In [96]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

# Spécifiez les chemins des quatre parties du CSV découpé
partie1 = "../Dataset/SBAnational_part_part1.csv"
partie2 = "../Dataset/SBAnational_part_part2.csv"
partie3 = "../Dataset/SBAnational_part_part3.csv"
partie4 = "../Dataset/SBAnational_part_part4.csv"

# Utilisez la fonction charger_csv pour obtenir un DataFrame avec toutes les données réassemblées
df = charger_csv(partie1, partie2, partie3, partie4)

/home/utilisateur/projet/Brief_Prêt/Model_pret/Model/../Module/loader_csv.py:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_partie4 = pd.read_csv(partie4)


In [97]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[6487124005, 3816854010, 9101293005, 310825400..."
1,Name,object,14,0.00,779583,"[S and W Enterprises, LASER FLASH, INC., Odom'..."
2,City,object,30,0.00,32581,"[VISTA, ATHENS, TONKAWA, OAKLAND, PEMBROKE (PE..."
3,State,object,14,0.00,51,"[NH, OH, LA, UT, TX]"
4,Zip,int64,0,0.00,33611,"[87114, 53089, 17601, 3079, 98103]"
5,Bank,object,1559,0.17,5802,"[STEARNS BK NATL ASSOC, JPMORGAN CHASE BANK NA..."
6,BankState,object,1566,0.17,56,"[CA, TX, NC]"
7,NAICS,int64,0,0.00,1312,"[811111, 621310, 444190, 453220, 713940]"
8,ApprovalDate,object,0,0.00,9859,"[4-Sep-07, 20-Jan-94, 18-Jan-02, 14-Dec-90, 9-..."
9,ApprovalFY,object,0,0.00,71,"[2004, 2000, 1990, 2004, 2003]"


In [98]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [99]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [100]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [101]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [102]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [103]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [104]:
df['FranchiseCode'] = df['FranchiseCode'].apply(lambda x: 1 if x >= 1 else 0)

In [105]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [106]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [107]:
df.shape

(675026, 13)

In [108]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[44-45, 31-33, 48-49]"
1,Term,int64,0,0.0,377,"[36, 180, 84, 131]"
2,NoEmp,int64,0,0.0,499,"[6, 4, 1, 10]"
3,NewExist,float64,0,0.0,2,"[2.0, 1.0]"
4,CreateJob,int64,0,0.0,216,"[0, 2]"
5,RetainedJob,int64,0,0.0,323,"[0, 22, 6, 3]"
6,FranchiseCode,int64,0,0.0,2,"[1, 0]"
7,UrbanRural,int64,0,0.0,3,[1]
8,RevLineCr,object,0,0.0,2,"[Y, N]"
9,LowDoc,object,0,0.0,2,[N]


In [109]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_0.csv


In [110]:
df = charger_csv(partie1, partie2, partie3, partie4)

/home/utilisateur/projet/Brief_Prêt/Model_pret/Model/../Module/loader_csv.py:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_partie4 = pd.read_csv(partie4)


In [111]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [112]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [113]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [114]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [115]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [116]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [117]:
df['FranchiseCode'] = df['FranchiseCode'].apply(lambda x: 1 if x >= 1 else 0)

In [118]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [119]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [120]:
df.shape

(460816, 13)

In [121]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[81, 62, 42, 54, 44-45]"
1,Term,int64,0,0.0,373,"[36, 70, 72, 84]"
2,NoEmp,int64,0,0.0,432,"[14, 3, 2, 5, 4]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,192,"[0, 1]"
5,RetainedJob,int64,0,0.0,276,"[3, 11, 0, 1]"
6,FranchiseCode,int64,0,0.0,2,"[1, 0]"
7,UrbanRural,int64,0,0.0,3,"[1, 0]"
8,RevLineCr,object,0,0.0,2,"[Y, N]"
9,LowDoc,object,0,0.0,2,[N]


In [122]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_1.csv


In [ ]:
# samples_bad_lol = df[(df['ChgOffDate'].notna())]
# df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
# samples_bad_lol['MIS_Status'].value_counts()

In [ ]:
# status_counts = df.loc[samples_bad_lol.index, 'MIS_Status'].value_counts()#.isna().sum() == 0

# print(status_counts)

In [ ]:
# samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
# df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
# df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)
# samples_bad['MIS_Status'].value_counts()

In [ ]:
# samples_bad['MIS_Status'].isna().sum()

In [ ]:
# status_counts = df.loc[samples_bad.index, 'MIS_Status'].value_counts()#.isna().sum() == 0

# status_counts